In [ ]:
import os
import numpy as np
import open3d as o3d
import copy

In [ ]:
storeData = []
nameFile = "../../data/Q4/LiDAR/0000"
for i in range(10, 87):
    x = nameFile + str(i) + ".bin"
    binPcd = np.fromfile(x, dtype=np.float32)
    binPcd = binPcd.reshape((-1, 4))[:, 0:3]
    storeData.append(binPcd)

storeData = np.array(storeData, dtype=object)

In [ ]:
odoFile = "../../data/Q4/odometry.txt"
filePtr = open(odoFile, "r")
matrix = []
for i in range(77):
    row = (filePtr.readline()).split()
    rowInMatrix= np.array(list(map(float, row)))
    rowInMatrix = np.reshape(rowInMatrix, (3, 4))
    matrix.append(rowInMatrix)


In [ ]:
lidToCam = np.array([[0,0,1,0],[-1,0,0,0], [0,-1,0,0],[0,0,0,1]])
lidToCam = np.transpose(lidToCam)

In [ ]:
allPoses = []
for i in range(77):
    numPoints = storeData[i].shape[0]
    currPose = []
    for j in range(numPoints):
        homoVec = np.array([storeData[i][j][0], storeData[i][j][1], storeData[i][j][2], 1])
        convertedVec = np.matmul(lidToCam, homoVec)
        currPose.append(convertedVec)
    allPoses.append(currPose)


In [ ]:
homoMatrix = []
for i in range(77):
    curr = np.array([[matrix[i][0][0], matrix[i][0][1], matrix[i][0][2], matrix[i][0][3]], [matrix[i][1][0], matrix[i][1][1], matrix[i][1][2], matrix[i][1][3]], [matrix[i][2][0], matrix[i][2][1], matrix[i][2][2], matrix[i][2][3]], [0,0,0,1]]) 
    homoMatrix.append(curr)
worldFrame = np.array([[0,0,1,0],[-1,0,0,0], [0,-1,0,0] ,[0,0,0,1]])
ctw = []
wtc = []
for i in range(77):
    ctw.append(np.matmul(worldFrame, homoMatrix[i]))
    wtc.append(np.linalg.inv(ctw[i]))

In [ ]:
inCommonFrame = []
for i in range(77):
    allPoses[i] = np.array(allPoses[i])
    extent = allPoses[i].shape[0]
    for j in range(extent):
        inCommonFrame.append(np.matmul(homoMatrix[i], allPoses[i][j])[0:3])


In [9]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(inCommonFrame)
# o3d.visualization.draw_geometries([pcd])

In [10]:
def convertCamera(x, pcd):
    o3d.visualization.draw_geometries([pcd.transform(wtc[x])])
